# Keras with MENext backend training routine

In [ ]:
import os
import sys
import subprocess
from pdb import set_trace
import gc
import time
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from IPython import display

import matplotlib.pyplot as plt

#!pip install pyfftw
import pyfftw
from pyfftw.interfaces.scipy_fftpack import rfft, irfft

#!pip install librosa
import librosa

#!pip install soundfile
import soundfile as sf

import pywt

# Setup

In [ ]:
# Data loading
# Contains full paths to files, 
# with 'simPath' column for merged audio (X)
# and 'crowdSegPath' for target (y)
train_df_path = './train.csv' 
test_df_path = './test.csv'
val_df_path = './val.csv'
audio_len = 65536 # length of audio clips

# Model
previous_weights = None#'./train1/model_5_weights-improvement-02-0.26.hdf5' # continue from previous training

# Training
epochs = 2000 # number of epochs to train
batch_size=16 # batch size for training
train_num = 1 # training results will be saved in the trainNum folder
model_num = 1 # model results will be saved as model_num file

# number of GPUs used for training. 
# For multiple GPUs: make sure ((num_train) % batch_size) / num_gpu == 0
num_gpus = 1

# Data transformation used for training
datatype='wavelet' #[wavelet, time (no transformation)]

# Saving results
train_dir = './train'
save_dir = '/data/output_data/'
wavelet_dir = '/data/simulated_wavelet10' # if training using wavelet, make sure this directory exists

# Data generator

In [ ]:
#import tensorflow as tf
import keras
from keras import Input, metrics, losses
from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import Input, Dense, Activation, Dropout, Flatten, BatchNormalization, Lambda
from keras.layers import Conv1D, Conv2DTranspose, MaxPooling1D, AveragePooling1D, UpSampling1D, UpSampling2D, K
from keras.layers import Concatenate, LeakyReLU
from keras.layers.convolutional import _UpSampling
from keras.legacy import interfaces
from keras.callbacks import ModelCheckpoint, EarlyStopping #, TensorBoard
from keras.regularizers import l1, l2, l1_l2
from keras.initializers import VarianceScaling
from keras.callbacks import CSVLogger
from keras.utils import multi_gpu_model

In [ ]:
# Second level decomposition, Haar
def time2wavelet(x):
        cA2, cD2, cD1 = pywt.wavedec(x, 'haar', mode='constant', level=2)
        w = np.concatenate((cA2, cD2, cD1)) # enhancing higher frequency space
        w = w / np.max(np.abs(w))
        return w

def wavelet2time(w, weights=[1, 1, 1]):
    coeffs = w[:16384], w[16384:32768], w[32768:]
    x = pywt.waverec(coeffs, 'haar')
    x = x / np.max(np.abs(x))
    return x

In [ ]:
# Generating data
class DataGenerator(keras.utils.Sequence):
    """Generate data for Keras"""
    def __init__(self, df, batch_size=50, dataset='train', shuffle=True, save_transform=False, datatype='time', 
                 input_dims=[32000], data_format='channels_first'):
        """
        * df: data frame that stores the meta data of the dataset. 
            The generator will iterate through this dataframe.
        * batch_size: batch sample of each iteration
        * dataset: use to label the dataset of the current generator
        * shuffle: whether or not shuffle the dataframe before itering over it. Default True!
        * datatype: type of data to load: 'time' or 'fft'
        * savefft: save the fft transformation used to train the data as a file to speed up training over epochs
        * input_dims: dimension of the input
        * duration: duration of the data to get (the last index). By default, get all data. 
        """
        self.batch_size = batch_size
        self.dims = input_dims
        self.dataset = dataset
        self.shuffle = shuffle
        self.datatype = datatype
        self.df = df
        self.save_transform = save_transform
        self.data_format = data_format
        self.on_epoch_end()
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        num_batches = self.df.shape[0] // self.batch_size + (self.df.shape[0] % self.batch_size > 0)
        return num_batches
    
    def __getitem__(self, index):
        """Generate one batch of data"""
        start_index = index * self.batch_size
        end_index = (index+1) * self.batch_size
        if end_index > self.df.shape[0]:
            end_index = None
        rows = self.df.index[start_index:end_index]
        
        X, y = self.__data_generation(rows)

        return X, y
    
    def on_epoch_end(self):
        """Update indexes after each epoch"""
        start_time = time.clock()
        if self.shuffle:
            self.df = self.df.sample(frac=1, replace=False).reset_index(drop=True)
    
    def __data_generation(self, rows):
        nrows = len(rows)
        # crowd + vice
        dims = [nrows] +  [1] + list(self.dims) \
                if self.data_format == 'channels_first' \
                else [nrows] +  list(self.dims) + [1] # crowd + noise
        X = np.empty(dims)
        # crowd and voice separately
        dims = [nrows] +  [1] + list(self.dims) \
                if self.data_format == 'channels_first' \
                else [nrows] +  list(self.dims) + [1] # crowd + noise
        y = np.empty(dims)
        
        func = self.__load_wavelet if self.datatype == 'wavelet' else self.__load_data
        #start_time = time.clock()
        for n, r in enumerate(rows):
            # Load crowd + voice file
            if self.data_format == 'channels_first':
                X[n, 0, :] = func(self.df.loc[r, 'simPath'])
                y[n, 0, :] = func(self.df.loc[r, 'crowdSegPath'])
            else:
                X[n, :, 0] = func(self.df.loc[r, 'simPath'])
                y[n, :, 0] = func(self.df.loc[r, 'crowdSegPath'])
        return X, y
    
    def __load_data(self, path=None, normalize=True, *args, **kwargs):
        y, fs = sf.read(path)
        # Take only the first channel
        y = y[:self.dims[0], 0]
        y[np.isnan(y)] = 0
        
        if normalize:
            y = 1.0/np.max(abs(y), axis=0) * y
            #y = (y - y.min(axis=0)) / (y.max(axis=0) - y.min(axis=0))
            
        return y
    
    def __load_wavelet(self, path=None, normalize=True):
        """
        Load the wavelet coefficients of the times series file.
        If not already exist, create it.
        normalize: normalize the time series to 0 to 1 before converting fft
        self.save_transform: [True|False] save the new fft files in a drive, given a path here.
        """
        wl_path = path.replace('.wav', '_wavelet.npz').replace('simulated_wav', 'simulated_wavelet')
        if not os.path.isfile(wl_path): # file specified but not there!
            # load file
            y, fs = sf.read(path)
            # Take only the first two seconds and first channel
            y = y[:self.dims[0], 0]
            y[np.isnan(y)] = 0
            #set_trace()
            if normalize: # -1 and 1
                y = 1.0/np.max(abs(y), axis=0) * y
                #y = (y - y.min(axis=0)) / (y.max(axis=0) - y.min(axis=0))

            w = time2wavelet(y)

            if self.save_transform:
                np.savez(wl_path, w=w)
        else: # already exists, simply load it
            w = np.load(wl_path)['w']
            
        return w
    
test_ = True
save_transform=True
start_fresh = True # recalculating any transformations
data_format = 'channels_first' # batch x channel x time; channel_last: batch x time x channel (slow on GPU)

# Start fresh
try:
    cmd = 'mkdir ' + wavelet_dir
    subprocess.call(cmd, shell=True)
except:
    pass
    
if start_fresh:
    cmd = 'rm -f ' + wavelet_dir +'/*.npz'
    subprocess.call(cmd, shell=True)

# Initialize data generator
if test_:
    df_train = pd.read_csv(train_df_path)
    df_test = pd.read_csv(test_df_path)
    df_val = pd.read_csv(val_df_path)
    train_generator = DataGenerator(df_train.reset_index(drop=True), batch_size=batch_size, shuffle=True, 
                                    save_transform=save_transform, dataset='train', input_dims=[audio_len],
                                    datatype=datatype, data_format=data_format)
    test_generator = DataGenerator(df_test.reset_index(drop=True), batch_size=batch_size, shuffle=False, 
                                   save_transform=save_transform, dataset='test', input_dims=[audio_len], 
                                   datatype=datatype, data_format=data_format)
    val_generator = DataGenerator(df_val.reset_index(drop=True), batch_size=batch_size, shuffle=False, 
                                  save_transform=save_transform, dataset='val', input_dims=[audio_len], 
                                  datatype=datatype,  data_format=data_format)

In [ ]:
X, y = train_generator.__getitem__(0)
print(X.shape)
print(y.shape)

# Define the denoising autoencoder with U-net structure

In [ ]:
# custom losses
def corr_loss(y_true, y_pred):
    r = corr_metric(y_true, y_pred)
    return 1 - K.square(r)
    
def corr_metric(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(xm * ym) #K.sum(tf.multiply(xm, ym))
    r_den = K.sqrt(K.sum(K.square(xm)) * K.sum(K.square(ym))) #K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / (r_den + 1E-12) # prevent nans

    r = K.maximum(K.minimum(r, 1.0), -1.0)
        
    return r

def corr_crowd_metric(y_true, y_pred): # channels_last, tf backend
    y_true_crowd = K.slice(y_true, [0, 0, 0], [-1, -1, 1])
    y_pred_crowd = K.slice(y_pred, [0, 0, 0], [-1, -1, 1]) 
    return corr_metric(y_true_crowd, y_pred_crowd)

def corr_voice_metric(y_true, y_pred): # channels_last, tf backend
    y_true_voice = K.slice(y_true, [0, 0, 1], [-1, -1, 1])
    y_pred_voice = K.slice(y_pred, [0, 0, 1], [-1, -1, 1])
    return corr_metric(y_true_voice, y_pred_voice)

def corr_crowd_mxnet(y_true, y_pred):
    if data_format == 'channels_last':
        y_true_crowd = Lambda(lambda x: x[:, :, 0])(y_true)
        y_pred_crowd = Lambda(lambda x: x[:, :, 0])(y_pred)
    else:
        y_true_crowd = Lambda(lambda x: x[:, 0, :])(y_true)
        y_pred_crowd = Lambda(lambda x: x[:, 0, :])(y_pred)
    return corr_metric(y_true_crowd, y_pred_crowd) # channels_first

def corr_voice_mxnet(y_true, y_pred):
    if data_format == 'channels_last':
        y_true_crowd = Lambda(lambda x: x[:, :, 1])(y_true)
        y_pred_crowd = Lambda(lambda x: x[:, :, 1])(y_pred)
    else:
        y_true_crowd = Lambda(lambda x: x[:, 1, :])(y_true)
        y_pred_crowd = Lambda(lambda x: x[:, 1, :])(y_pred)
    return corr_metric(y_true_crowd, y_pred_crowd) # channels_first
    

In [ ]:
class UpSampling1D_2(_UpSampling):
    """Upsampling layer for 1D inputs.
    Repeats each temporal step `size` times along the time axis.
    # Arguments
        size: integer. Upsampling factor.
    # Input shape
        3D tensor with shape: `(batch, steps, features)`.
    # Output shape
        3D tensor with shape: `(batch, upsampled_steps, features)`.
    """

    @interfaces.legacy_upsampling1d_support
    def __init__(self, size=2, **kwargs):
        super(UpSampling1D_2, self).__init__((int(size),), 'channels_first', **kwargs)

    def call(self, inputs):
        output = K.repeat_elements(inputs, self.size[0], axis=2)
        return output

    def get_config(self):
        config = super(UpSampling1D_2, self).get_config()
        config['size'] = self.size[0]
        config.pop('data_format')
        return config

def UpSampling1D_interpolate(input_tensor, size, interpolation='bilinear', data_format='channels_last'):
    x = Lambda(lambda x: K.expand_dims(x, axis=1))(input_tensor)
    x = UpSampling2D(size=(size, 1), data_format=data_format, interpolation=interpolation)(x)
    x = Lambda(lambda x: K.squeeze(x, axis=1))(x)
    return x


In [ ]:
K.clear_session()
alpha_leakyrelu = 0.1
axis = 1 if data_format == 'channels_first' else 2

def conv_block(x, filter_size=64, nconv='double', kernel_size=3, padding='same'):
    if nconv=='double':
        conv = Conv1D(filters=filter_size, kernel_size=kernel_size, padding=padding, activation=None, 
                     data_format=data_format)(x)
        #conv = BatchNormalization(axis=axis)(conv)
        conv = LeakyReLU(alpha_leakyrelu)(conv)
    else:
        conv = x
    conv = Conv1D(filters=filter_size, kernel_size=kernel_size, padding=padding, activation=None, 
                 data_format=data_format)(conv)
    conv = BatchNormalization(axis=axis)(conv)
    conv = LeakyReLU(alpha_leakyrelu)(conv)
    return conv

def build_generator():
    inputs = Input(shape=(1, audio_len)) if  data_format == 'channels_first' \
                                         else Input(shape=(audio_len, 1))
    # Encoder
    conv1 = conv_block(inputs, 64, 'double') # L x 64
    pool1 = MaxPooling1D(pool_size=2, padding='same', data_format=data_format)(conv1) # L/2 x 64
    
    conv2 = conv_block(pool1, 128, 'double') # L/2 x 128
    #conv2 = Dropout(0.5)(conv2) # drop out layer regularization
    pool2 = MaxPooling1D(pool_size=2, padding='same', data_format=data_format)(conv2) # L/4 x 128
    
    conv3 = conv_block(pool2, 256, 'double') # L/4 x 256
    #conv3 = Dropout(0.5)(conv3) # drop out layer regularization
    pool3 = MaxPooling1D(pool_size=2, padding='same', data_format=data_format)(conv3) # L/8 x 256
    
    conv4 = conv_block(pool3, 256, 'double') # L/8 x 256
    #conv4 = Dropout(0.5)(conv4) # drop out layer regularization

    
    # Decoder
    upsamp1 = UpSampling1D(size=2)(conv4) if data_format == 'channels_last' \
                                            else UpSampling1D_2(size=2)(conv4) # L/4 x 256
    deconv1 = Concatenate(axis=axis)([upsamp1, conv3]) # U: L/4 x 512 (2 x 256)
    deconv1 = conv_block(deconv1, 128, 'double') # L/4 x 128
    
    
    upsamp2 = UpSampling1D(size=2)(deconv1) if data_format == 'channels_last' \
                                            else UpSampling1D_2(size=2)(deconv1) # L/2 x 128
    deconv2 = Concatenate(axis=axis)([upsamp2, conv2]) # U: L/2 x 256 (2 x 128)
    deconv2 = conv_block(deconv2, 64, 'double') # L/2 x 64
    
    upsamp3 = UpSampling1D(size=2)(deconv2) if data_format == 'channels_last' \
                                            else UpSampling1D_2(size=2)(deconv2) # L x 64
    deconv3 = Concatenate(axis=axis)([upsamp3, conv1]) # U: L x 129 (2 x 64 + 1)
    deconv3 = Conv1D(filters=16, kernel_size=3, padding='same', activation=None, 
                  data_format=data_format)(deconv3)  # L x 16
    deconv3 = LeakyReLU(alpha_leakyrelu)(deconv3)
    
    deconv4 = Concatenate(axis=axis)([deconv3, inputs]) # U: L x 17  (16 + 1)
    deconv4 = Conv1D(filters=1, kernel_size=3, padding='same', activation='tanh', 
                  data_format=data_format)(deconv4)  # L x 2
    
    model = Model(inputs, deconv4)
    
    return model

def backend_agnostic_compile(model, loss, optimizer, metrics, num_gpus=8):
    if keras.backend._backend == 'mxnet':
        gpu_list = ["gpu(%d)" % i for i in range(num_gpus)]
        model.compile(loss=loss,
            optimizer=optimizer,
            metrics=metrics, 
            context = gpu_list)
    else:
        model = multi_gpu_model(model, gpus=num_gpus)
        model.compile(optimizer=optimizer, loss=loss,
                       metrics=metrics)
        
    return model

model = build_generator()
opt = keras.optimizers.Adam(lr=0.0001, decay=1e-5)

# load previous model
if previous_weights:
    model.load_weights(previous_weights)

if num_gpus > 1:
    # check if the num batches can be evenly distributed
    if (df_train.shape[0] % batch_size) / num_gpus !=0 or \
        (df_test.shape[0] % batch_size) / num_gpus !=0 or \
        (df_val.shape[0] % batch_size) / num_gpus !=0:
        raise(BaseException('Cannot train on multile gpus!\nMake sure ((num_train) % batch_size) / num_gpu == 0'))
    else:
        model = backend_agnostic_compile(model, optimizer=opt, 
                         loss = corr_loss, #losses.mean_squared_error, 
                         metrics =[corr_crowd_mxnet, metrics.mean_squared_error], #corr_crowd_metric, corr_voice_metric,
                         num_gpus=num_gpus)
else:
    model.compile(optimizer=opt,
                  loss=losses.mean_squared_error, # with regularization added on
                  metrics=[corr_crowd_mxnet, metrics.mean_squared_error])

print(model.summary())

# Training

In [ ]:
try:
    os.mkdir(os.path.join(train_dir,"train"+str(train_num)))
except:
    pass
checkfilepath = os.path.join(train_dir, "train"+str(train_num)+"/model_"+str(model_num)+\
            "_weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5")
checkpoint = ModelCheckpoint(checkfilepath, monitor='val_loss', verbose=0, save_best_only=True)
#tensorboard = TensorBoard(log_dir="./train"+str(train_num), histogram_freq=5,write_graph=True,write_images=True)
csv_logger = CSVLogger(os.path.join(train_dir, 'train'+str(train_num)+'/model_'+str(model_num)+'_training.log'))
early_stop = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit_generator(train_generator,
                              epochs=epochs,
                              validation_data=val_generator, 
                              callbacks = [csv_logger, checkpoint, early_stop],# tensorboard, 
                              verbose=1)

In [ ]:
# Serialize the model to json
model.save(os.path.join(train_dir, 'train'+str(train_num)+'/CNN_model_'+str(model_num)+'_corr_loss.hdf5'))
model_json = model.to_json()
with open(os.path.join(train_dir, "train"+str(train_num)+"/CNN_model_"+str(model_num)+\
                       "_corr_loss_specification.json"), "w") as json_file:
    json_file.write(model_json)
#Serialize weights to HDF5
model.save_weights(os.path.join(train_dir, "train"+str(train_num)+\
                                "/CNN_model_"+str(model_num)+"_corr_loss_weights.h5"))
# save history
try:
    with open( os.path.join(train_dir, 'train'+str(train_num)+'/history_model_'+\
                            str(model_num)+'_corr_loss.json'), 'w') as f:
        json.dump(history.history, f)
except:
    pass


In [ ]:
# Do some plots
fig, axs = plt.subplots(1, 2)
history_df = pd.read_csv(os.path.join(train_dir, 'train'+str(train_num)+\
                                      '/model_'+str(model_num)+'_training.log'))
axs[0].plot(history_df['loss'])
axs[0].plot(history_df['val_loss'])
axs[0].set_title('loss')
axs[1].plot(history_df['corr_crowd_mxnet'])
axs[1].plot(history_df['val_corr_crowd_mxnet'])
axs[1].set_title('crowd correlation')
fig.set_size_inches(12, 3)

# Save outputs on the test set

In [ ]:
def load_autoencoder_model(model_file, weight_file):
    # load json and create model
    json_file = open(model_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(weight_file)
    
    #model = load_model(model_file) if saved as hdf5 directly
    
    opt = keras.optimizers.Adam(lr=0.1, decay=0.01)

    model.compile(optimizer=opt,
              loss=K.binary_crossentropy,
              metrics=[metrics.binary_accuracy, metrics.mean_squared_error])
    
    return model

def predict_audio(model, audios):
    predicted_audio_segments = model.predict(audios)
    return predicted_audio_segments#.squeeze()

In [ ]:
# Write Files DWT domain
fs = 16000
count = 0

df = pd.DataFrame({'id':np.nan, 'mse':np.nan, 'corr':np.nan}, index=[])
from scipy.signal import butter, filtfilt
def filter_audio(audio, freq=[3500/8000, 4300/8000]):
    b, a = butter(5, freq, 'bandstop')
    audio = filtfilt(b, a, audio)
    return audio

for i in range(len(test_generator)):
    X, y = test_generator.__getitem__(i)
    y_pred = predict_audio(model, X)
        
    for index in range(X.shape[0]):
        X_merge = X[index, :, 0] if data_format == 'channels_last' else X[index, 0, :]
        y_crowd = y[index, :, 0] if data_format == 'channels_last' else y[index, 0, :]
        y_crowd_pred = y_pred[index, :, 0] if data_format == 'channels_last' else y_pred[index, 0, :]

        arrays_dwt = [X_merge, y_crowd,  y_crowd_pred]#, y_voice, y_voice_pred]
        wave_types = ['merge', 'crowd', 'recon']
        arrays = [[]] * 3

        for n, (k, w) in enumerate(zip(arrays_dwt, wave_types)):
            out_wav = wavelet2time(k)
            out_wav = filter_audio(out_wav)
            arrays[n] = out_wav
            librosa.output.write_wav(os.path.join(save_dir, "{:04d}_{}.wav".format(count, w)), out_wav, 16000)
        
        df.loc[count, 'id'] = "{:04d}".format(count)
        df.loc[count, 'mse'] = np.mean((arrays[1] - arrays[2])**2)
        df.loc[count, 'corr'] = np.corrcoef(arrays[1], arrays[2])[0, 1]
        count = count + 1
            

In [ ]:
df_test_out = pd.concat([df_test, df], axis=1)
df_test_out.to_csv(os.path.join(save_dir, 'test_results.csv'), index=False)

# Optional Further Decomposition Using ICA
Can sometimes enhance the final separation. Need to manually determine which source is the target.

In [ ]:
"""
from sklearn.decomposition import FastICA

out_mixed = np.c_[arrays[0], arrays[2]]
fica = FastICA(n_components=2)
out_unmixed = fica.fit_transform(out_mixed)

# Do some filtering before display
out_unmixed_filtered = filter_audio(out_unmixed)

print('Original')
display.display(display.Audio(arrays[0], rate=fs))
print('Source 1')
display.display(display.Audio(out_unmixed_filtered[:, 0], rate=fs))
print('Source 2')
display.display(display.Audio(out_unmixed_filtered[:, 1], rate=fs))
"""